# Machine Learning Classification

*Author: Evan Carey*

*Copyright 2017, BH Analytics, LLC*

## Overview

The purpose of this section is to go over machine learning, specifically classification, in the context of python (the scikit-learn module). We will include some general concepts of machine learning as well as the specifics of a few different classification algorithms. For further reading, I highly recommend the free ebook titled 'Introduction to Statistical Learning' by Gareth James. A quick web search should find this book near the top of the search results. For even more in-depth coverage of machine learning algorithms, I recommend the book  'Elements of Statistical Learning' by Trevor Hastie (also free online). 

## Classification

In the case where our outcome (target) variable is discrete with a limited number of possible values, we can use classification algorithms to predict the outcome. Imagine a binary outcome with values of 'Yes' and 'No'. We are interested in predicting the probability that the outcome is either 'Yes' or 'No'. It is also possible to predict outcomes with more than two possible values, but we will focus on the binary case here. 

## Libraries

In [1]:
## Import Modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn import datasets
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
import seaborn as sns

In [2]:
## Enable inline plotting for graphics
%matplotlib inline

In [3]:
## Get Version information
print(sys.version)
print("Pandas version: {0}".format(pd.__version__))
print("Matplotlib version: {0}".format(matplotlib.__version__))
print("Numpy version: {0}".format(np.__version__))
print("SciKitLearn version: {0}".format(sklearn.__version__))
print("Statsmodels version: {0}".format(statsmodels.__version__))

3.6.0 |Anaconda custom (64-bit)| (default, Dec 23 2016, 11:57:41) [MSC v.1900 64 bit (AMD64)]
Pandas version: 0.19.2
Matplotlib version: 2.0.0
Numpy version: 1.11.3
SciKitLearn version: 0.18.1
Statsmodels version: 0.6.1


## Check your working directory

Subsequent sessions may require you to identify and update your working directory so paths correctly point at the downloaded data files. You can check your working directory like so:

In [4]:
## Check/set your working directory
print("My working directory:\n" + os.getcwd())
# Set Working Directory (if needed)
os.chdir(r"C:\Users\evan\Dropbox\BH_Katie_Share\Goldman_Sachs")
print("My new working directory:\n" + os.getcwd())

My working directory:
J:\Sync\Work\BHAnalytics\Python\01_Current_Python\notebooks
My new working directory:
C:\Users\evan\Dropbox\BH_Katie_Share\Goldman_Sachs


## Machine Learning

* Machine learning is the science of programming algorithms to learn from data.
* Machine learning is focused on prediction – Predictive Modeling
* Machine learning typically uses finite sample properties to optimize models
* Traditional statistics focuses on process inference -statistical inference
* Traditional statistics often use asymptotic properties to make inference
* In practice, there is much overlap between the disciplines...

## Machine Learning Terminology

* Target: what I want to predict
* Inputs: The variables I am using to predict the target
* Samples: The rows of the dataframe
* Unsupervised Learning: Algorithms without a target
  * Clustering
  * Dimensionality Reduction
* Supervised Learning: Algorithms using a target
  * Classication: Target is discrete
  * Regression: Target is continuous

## Variance, Bias, Over-fitting

* Variance refers to how variables your estimates are
* Bias refers to how accurate your estimates are
* Think of shooting arrows at a target: Low bias means on average, you get close to the center. Low variance means your arrows are all close together
* There is often a tradeoff between bias and variance.
* Over-fitting our data: too many attempts at fitting leads to a great fit...but only for this data?


## Avoiding Overfitting

* Incorporate your topical knowledge when selecting features
* The best models come from careful feature selection rather than automated decisions.
* Remove redundant variables (highly correlated)
* Censor values in your data you know to be inaccurate (Careful data cleaning)
* Identify structure in your data and use it appropriately

## Cross Validation

* Create training and test datasets
  * Random splits
  * Split by year
  * Split by cluster
* Run this several times
* Be wary of implausibly great results - may have overfit the data

## Steps to Machine Learning

* Identify your question to be answered: classification, regression, or clustering?
* What data is available?
* Do you have targets (supervised) ?
* What features make the most sense in the context of your problem?
* Clean your data for unlikely values, examine your dataset
* Identify models that correspond to your structural requirements
* Distribution of the target (how many levels, what is the domain of the numbers if regression)

## Rand Health Experiment Dataset

We will use the Rand Health data that is included with statsmodels for the applied section of machine learning classification. You can call the data into memory using the following code:

In [5]:
import statsmodels
## Import Data
rand_hie = statsmodels.datasets.randhie.load_pandas().data

We can examine information about the data like so:

In [6]:
print(statsmodels.datasets.randhie.NOTE)

::

    Number of observations - 20,190
    Number of variables - 10
    Variable name definitions::

        mdvis   - Number of outpatient visits to an MD
        lncoins - ln(coinsurance + 1), 0 <= coninsurance <= 100
        idp     - 1 if individual deductible plan, 0 otherwise
        lpi     - ln(max(1, annual participation incentive payment))
        fmde    - 0 if idp = 1; ln(max(1, MDE/(0.01 coinsurance))) otherwise
        physlm  - 1 if the person has a physical limitation
        disea   - number of chronic diseases
        hlthg   - 1 if self-rated health is good
        hlthf   - 1 if self-rated health is fair
        hlthp   - 1 if self-rated health is poor
        (Omitted category is excellent self-rated health)



As always, let's first check the top and bottom of the data and some basic descriptions:

In [7]:
rand_hie.head()

mdvis  lncoins  idp       lpi  fmde  physlm     disea  hlthg  hlthf  hlthp
0      0  4.61512    1  6.907755   0.0     0.0  13.73189      1      0      0
1      2  4.61512    1  6.907755   0.0     0.0  13.73189      1      0      0
2      0  4.61512    1  6.907755   0.0     0.0  13.73189      1      0      0
3      0  4.61512    1  6.907755   0.0     0.0  13.73189      1      0      0
4      0  4.61512    1  6.907755   0.0     0.0  13.73189      1      0      0

In [8]:
rand_hie.tail()

mdvis   lncoins  idp       lpi      fmde    physlm     disea  hlthg  \
20185      2  0.000000    0  5.377498  0.000000  0.144292  10.57626      0   
20186      0  0.000000    0  5.377498  0.000000  0.144292  10.57626      0   
20187      8  3.258096    0  6.874819  8.006368  0.144292  10.57626      0   
20188      8  3.258096    0  5.156178  6.542472  0.144292  10.57626      0   
20189      6  3.258096    0  6.620073  8.006368  0.144292  10.57626      0   

       hlthf  hlthp  
20185      0      0  
20186      0      0  
20187      0      0  
20188      0      0  
20189      0      0

In [9]:
rand_hie.describe()

mdvis       lncoins           idp           lpi          fmde  \
count  20190.000000  20190.000000  20190.000000  20190.000000  20190.000000   
mean       2.860426      1.774071      0.259980      4.707894      4.029524   
std        4.504365      1.983272      0.438634      2.697840      3.471353   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      4.063885      0.000000   
50%        1.000000      0.000000      0.000000      6.109248      6.093520   
75%        4.000000      3.931826      1.000000      6.620073      6.959049   
max       77.000000      4.615120      1.000000      7.163699      8.294049   

             physlm         disea         hlthg         hlthf         hlthp  
count  20190.000000  20190.000000  20190.000000  20190.000000  20190.000000  
mean       0.123500     11.244492      0.362011      0.077266      0.014958  
std        0.322016      6.741449      0.480594      0.267020      0.121387  
min        0.000000      0.000000      0.000000      0.000000      0.000000  
25%        0.000000      6.900000      0.000000      0.000000      0.000000  
50%        0.000000     10.576260      0.000000      0.000000      0.000000  
75%        0.000000     13.731890      1.000000      0.000000      0.000000  
max        1.000000     58.600000      1.000000      1.000000      1.000000

In [10]:
rand_hie.dtypes

mdvis        int64
lncoins    float64
idp          int64
lpi        float64
fmde       float64
physlm     float64
disea      float64
hlthg        int64
hlthf        int64
hlthp        int64
dtype: object

## Create Outcome Variable

For this dataset, we will create a new variable to indicate it mdvis is greater than 0. This will be our outcome variable.

In [11]:
rand_hie['mdvis_YN'] = np.where(rand_hie['mdvis']>0,'Yes','No')

In [12]:
rand_hie.mdvis_YN.value_counts()

Yes    13882
No      6308
Name: mdvis_YN, dtype: int64

In [13]:
rand_hie.mdvis_YN.value_counts(normalize = True)

Yes    0.687568
No     0.312432
Name: mdvis_YN, dtype: float64

## Workflow into scikit-learn

* There are a number of possible ways to prepare data for modeling in scikit-learn. 
* You must end up with a numeric ndarray of inputs (X) and a numeric ndarray matrix of the target (Y)
* I prefer the following workflow:
  * We use pandas to import and clean data
  * We use Patsy to create the X and Y ndarrays
  * Using categorical transformations (dummy coding) as needed
  * Also can generate non-linear terms including splines
  * Use scikit-learn for machine learning

## Use Patsy to Create the Model Matrices

We typically start out with a pandas dataframe for manipulation purposes, then we will use this dataframe as the input to the machine learning library. I created a pandas dataframe above to replicate this process. We will use the dmatrices function from the patsy library to easily generate the design matrices for the machine learning algorithms. 

In [14]:
## Create formula for all variables in model
vars_remove = ['mdvis','mdvis_YN']
vars_left = set(rand_hie.columns) - set(vars_remove)
formula = "mdvis_YN ~ " + " + ".join(vars_left)
formula

'mdvis_YN ~ hlthg + hlthf + idp + disea + fmde + physlm + lpi + hlthp + lncoins'

In [15]:
## use Patsy to create model matrices
Y,X = dmatrices(formula,
                rand_hie)

In [16]:
Y

DesignMatrix with shape (20190, 2)
  mdvis_YN[No]  mdvis_YN[Yes]
             1              0
             0              1
             1              0
             1              0
             1              0
             1              0
             1              0
             0              1
             1              0
             1              0
             1              0
             0              1
             1              0
             1              0
             1              0
             0              1
             0              1
             1              0
             1              0
             1              0
             0              1
             1              0
             1              0
             1              0
             1              0
             0              1
             1              0
             0              1
             0              1
             0              1
  [20160 rows omitted]
  Terms:
   

In [17]:
X

DesignMatrix with shape (20190, 10)
  Columns:
    ['Intercept',
     'hlthg',
     'hlthf',
     'idp',
     'disea',
     'fmde',
     'physlm',
     'lpi',
     'hlthp',
     'lncoins']
  Terms:
    'Intercept' (column 0)
    'hlthg' (column 1)
    'hlthf' (column 2)
    'idp' (column 3)
    'disea' (column 4)
    'fmde' (column 5)
    'physlm' (column 6)
    'lpi' (column 7)
    'hlthp' (column 8)
    'lncoins' (column 9)
  (to view full data, use np.asarray(this_obj))

## Split into Testing and Training Samples

* The first step is to set aside a test sample of data that will allow us to examine the quality of our models. This protects against overfitting. 
* We can use “tuple unpacking” to assign the values (very pythonic :)
* We can assign a random seed (state) and fraction to split.

In [18]:
## Split Data into training and sample
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.25,
                                                    random_state=42)

## Confirm the Output Dimensions

* We can confirm the dimensions of the data are the same within test and train
* The proportion should also be close to the test_size argument. 

In [19]:
## Confirm dimensions
X_train.shape

(15142, 10)

In [20]:
X_test.shape

(5048, 10)

In [21]:
y_train.shape

(15142, 2)

In [22]:
y_test.shape

(5048, 2)

In [23]:
## Examine the shape of X
X_test.shape

(5048, 10)

## Choose the Outcome

Notice the formula split Y into two columns, one for yes and the inverse for no. 

In [24]:
y_test

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [25]:
y_test.mean(0)

array([ 0.30467512,  0.69532488])

We can see that the first column is the No, and the second column is the yes. Let's replace this with a single column to avoid issues later. You can change this line of code if you want to predict the other outcome.

In [26]:
y_test = y_test[:,0]
y_train = y_train[:,0]

## First Model: Logistic Regression

* We will start with a basic logistic regression model.
* The flow will be similar for all other models:
* Call and save model object with initial parameters, then call the fit() method, then call other summary methods post fit. 

In [27]:
## import linear model
from sklearn import linear_model
## Define model parameters
clf = linear_model.LogisticRegression()
## fit model using data with .fit
clf.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

How do we know if this is a good model? Let's make predictions, is this a good model? Which parameters are most important?

In [28]:
## Make predictions on test dataset
clf.predict(X_test)

array([ 0.,  1.,  0., ...,  0.,  0.,  0.])

## Model Summaries

* We can extract the model coefficients with the .coef_ attribute.

In [29]:
## Get coefficients
clf.coef_

array([[-0.19007506,  0.13067647,  0.34924831,  0.63985638, -0.06207179,
         0.05805466, -0.20681387, -0.10443719,  0.127628  ,  0.15607538]])

## Assessing the Model Score (accuracy)

* The concept of error is a bit different for a binary outcome than the continuous case. 
* We can imagine error is the number of incorrect predictions. 

In [30]:
## get mean accuracy
clf.score(X_test,y_test) 

0.70423930269413626

In this case, our model is accurate 70% of the time! We can understand what that means by looking at the predictions against the actual outcomes. This is called a confusion matrix.

In [31]:
## get confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,clf.predict(X_train))

array([[9978,  394],
       [4262,  508]])

In [32]:
## get classification metrics
print(sklearn.metrics.classification_report(y_train,
                                            clf.predict(X_train)))

             precision    recall  f1-score   support

        0.0       0.70      0.96      0.81     10372
        1.0       0.56      0.11      0.18      4770

avg / total       0.66      0.69      0.61     15142



In [33]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,clf.predict(X_train))

0.69251089684321754

In [34]:
## Get kappa
sklearn.metrics.cohen_kappa_score(y_train,clf.predict(X_train))

0.087722073286837232

## Keep Track of Scores Across Models

I am going to write a small function that will print the scores from a dict so we can compare the models. 

In [35]:
## Create dict to store all these results:
result_scores = {}
## Score the Model on Training and Testing Set
result_scores['Logistic'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [36]:
## Create Function to Print Results
def get_results(x1):
    print("\n{0:20}   {1:4}    {2:4}".format('Model','Train','Test'))
    print('----------------------------------')
    for i in x1.keys():
        print("{0:20}   {1:<6.4}   {2:<6.4}".format(i,x1[i][0],x1[i][1]))

In [37]:
get_results(result_scores)


Model                  Train    Test
----------------------------------
Logistic               0.6925   0.7042


## Comparison to the Null Model

Is that a good score for accuracy? Compared to what? We can consider a null model of simply predicting the most frequent class as a base model. Without any other information, I may predict based simply on the distribution of the outcome.

In [38]:
## Null information rate
y_test.mean()

array(0.30467511885895404)

Scikitlearn has a built in dummy classifier that works similarly:

In [39]:
## Dummy classifier
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent',
                      random_state=0)
clf.fit(X_train, y_train)
clf.score(X_train, y_train)  

0.68498216880200768

In [40]:
clf.score(X_test, y_test)  

0.69532488114104596

In [41]:
## Score the Model on Training and Testing Set
result_scores['Mean'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [42]:
get_results(result_scores)


Model                  Train    Test
----------------------------------
Logistic               0.6925   0.7042
Mean                   0.685    0.6953


## Logistic regression with L1 penalty

A different form of penalized logistic regression is to implement an L1 penalty. This is called a LASSO. We can easily do so like this:

In [43]:
## Logistic Regression with l1 penalty
clf = linear_model.LogisticRegression(penalty='l1', C=0.1)
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

array([[9980,  392],
       [4269,  501]])

In [44]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,clf.predict(X_train))

0.69218068947298905

In [45]:
## Get kappa
sklearn.metrics.cohen_kappa_score(y_train,clf.predict(X_train))

0.086146191568278851

In [46]:
## get classification metrics
print(sklearn.metrics.classification_report(y_train,clf.predict(X_train)))

             precision    recall  f1-score   support

        0.0       0.70      0.96      0.81     10372
        1.0       0.56      0.11      0.18      4770

avg / total       0.66      0.69      0.61     15142



In [47]:
## get classification metrics on test
print(sklearn.metrics.classification_report(y_test,clf.predict(X_test)))

             precision    recall  f1-score   support

        0.0       0.71      0.96      0.82      3510
        1.0       0.58      0.11      0.19      1538

avg / total       0.67      0.70      0.63      5048



In [48]:
## Score the Model on Training and Testing Set
result_scores['Logistic_L1'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [49]:
get_results(result_scores)


Model                  Train    Test
----------------------------------
Logistic               0.6925   0.7042
Mean                   0.685    0.6953
Logistic_L1            0.6922   0.7046


## Scaling / Pipeline

* We should consider scaling when we use shrinkage methods. 
* We can construct a pipeline to avoid having to apply the same transformation over and over again.
* We must use the StandardScaler() function here.

In [50]:
## LASSO regression, set alpha
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
clf = linear_model.LogisticRegression(C=0.1)
scaler = preprocessing.StandardScaler().fit(X_train)
pipe1 = Pipeline([("scale", scaler),
                     ("LASSO", clf)])
pipe1.fit(X_train,y_train)

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LASSO', LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [51]:
## Score on training data
pipe1.score(X_train,y_train) 

0.69237881389512612

In [52]:
## Score on testing data
pipe1.score(X_test,y_test) 

0.70404120443740092

In [53]:
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_1'] = \
            (pipe1.score(X_train,y_train),
             pipe1.score(X_test,y_test))

In [54]:
get_results(result_scores)


Model                  Train    Test
----------------------------------
Logistic               0.6925   0.7042
Mean                   0.685    0.6953
Logistic_L1            0.6922   0.7046
Logistic_L1_1          0.6924   0.704 


## Selecting Parameters via Cross Validation

For the LASSO regression, the primary parameter of interest is C. We can select the shrinkage parameter (C) through cross-validation. The default cross validation strategy is called 'leave one out'. We can also select a specific number of folds and do k-folds cross validation. 

In [55]:
## Select the alpha through cross validation (leave one out)
clf = linear_model.LogisticRegressionCV(Cs=[0.01, 0.5, 0.1, 0.15, 0.2, 0.5, 1])
scaler = preprocessing.StandardScaler()
pipe2 = Pipeline([("scale", scaler),
                     ("LASSO", clf)])
pipe2.fit(X_train,y_train)

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LASSO', LogisticRegressionCV(Cs=[0.01, 0.5, 0.1, 0.15, 0.2, 0.5, 1],
           class_weight=None, cv=None, dual=False, fit_intercept=True,
           intercept_scaling=1.0, max_iter=100, multi_class='ovr',
           n_jobs=1, penalty='l2', random_state=None, refit=True,
           scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

In [56]:
## Examine the score of the model on training data
pipe2.score(X_train,y_train) 

0.69323735305772027

In [57]:
## Score on testing data
pipe2.score(X_test,y_test) 

0.70404120443740092

In [58]:
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_C'] = \
            (pipe2.score(X_train,y_train),
             pipe2.score(X_test,y_test))

In [59]:
get_results(result_scores)


Model                  Train    Test
----------------------------------
Logistic               0.6925   0.7042
Mean                   0.685    0.6953
Logistic_L1            0.6922   0.7046
Logistic_L1_1          0.6924   0.704 
Logistic_L1_C          0.6932   0.704 


## Implementing Polynomial and Interaction Terms

* We can add polynomials as well as interactions using PolynomialFeatures. 
* We must give the degree argument. Polynomials up to that degree will be considered, and interactions between d-1 terms.
* This is a lot of parameters added into the model! That is why we are using LASSO to shrink some and avoid overfitting…

In [60]:
## try multiple polynomials with a LASSO
## use pipeline for pre-processing
## Select the alpha by 10 fold cross validation
from sklearn.preprocessing import PolynomialFeatures
clf = linear_model.LogisticRegressionCV(Cs=[0.1, 0.15, 0.2, 0.5,1],
                                        cv=10)
scaler = preprocessing.StandardScaler()
poly_feat = PolynomialFeatures(degree=2,include_bias=False)
pipe6 = Pipeline([("scale", scaler),
                  ("poly",poly_feat),
                  ("lasso", clf)])
pipe6.fit(X_train,np.ravel(y_train))

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('lasso', LogisticRegressionCV(Cs=[0.1, 0.15, 0.2, 0.5, 1], class_weight=None, cv=10,
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0))])

In [61]:
## Examine the score of the model on training data
pipe6.score(X_train,y_train) 

0.70307753269052964

In [62]:
## Score on testing data
pipe6.score(X_test,y_test) 

0.71038034865293187

In [63]:
## Examine the coefficients
pipe6.named_steps['lasso'].coef_

array([[  0.00000000e+00,   3.52394665e-02,   7.81870525e-03,
          1.83026792e-02,  -4.12952639e-01,   3.22022257e-01,
          8.97003466e-01,  -6.53124719e-01,  -2.18935529e-05,
          1.41233401e-03,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.90813148e-02,
         -1.57611309e-02,  -1.99543808e-02,  -4.73987827e-02,
         -1.64703754e-02,  -6.56682683e-02,   1.01203000e-02,
         -4.13830315e-03,   1.91014966e-02,   2.34366868e-02,
         -2.92393527e-02,  -4.94926214e-02,   2.09964492e-03,
         -3.12693987e-02,   5.49576432e-03,  -8.95295200e-04,
         -3.62071823e-02,   1.89626279e-02,   6.62356490e-02,
         -1.28815989e-01,  -2.21057944e-02,  -7.11621191e-02,
          8.93839932e-03,  -1.50605069e-02,   1.10691656e-02,
         -9.95036517e-02,   6.16157798e-02,  -7.54885159e-03,
        

In [64]:
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_poly'] = \
            (pipe6.score(X_train,y_train),
             pipe6.score(X_test,y_test))

In [65]:
get_results(result_scores)


Model                  Train    Test
----------------------------------
Logistic               0.6925   0.7042
Mean                   0.685    0.6953
Logistic_L1            0.6922   0.7046
Logistic_L1_1          0.6924   0.704 
Logistic_L1_C          0.6932   0.704 
Logistic_L1_poly       0.7031   0.7104


## Random forest

Another possible classification routine we can use is a random forest (an ensemble of decision trees). This algorithm is very sensitive to overfitting a single sample, so I have carefully chosen a few non-default parameters:

In [66]:
#### Fit Random Forest
## Random Forests
from sklearn import ensemble
clf = ensemble.RandomForestClassifier(n_estimators=20, 
                                      max_depth=None,
                                      min_samples_split=200,
                                      random_state=0)
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

array([[9752,  620],
       [3493, 1277]])

In [67]:
## Score the Model on Training and Testing Set
result_scores['RandomForest'] = \
            (clf.score(X_train,y_train),
             clf.score(X_test,y_test))

In [68]:
get_results(result_scores)


Model                  Train    Test
----------------------------------
Logistic               0.6925   0.7042
Mean                   0.685    0.6953
Logistic_L1            0.6922   0.7046
Logistic_L1_1          0.6924   0.704 
Logistic_L1_C          0.6932   0.704 
Logistic_L1_poly       0.7031   0.7104
RandomForest           0.7284   0.7324


## Support vector machine

Another reasonable choice for a classifer would be a support vector machine. We can easily implement this as well:

In [69]:
## Support Vector Machine (classification)
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

array([[9843,  529],
       [3775,  995]])

In [70]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,clf.predict(X_train))

0.71575749570730418

In [71]:
## Get kappa
sklearn.metrics.cohen_kappa_score(y_train,clf.predict(X_train))

0.19307448686672712

In [72]:
## get classification metrics
print(sklearn.metrics.classification_report(y_train,clf.predict(X_train)))

             precision    recall  f1-score   support

        0.0       0.72      0.95      0.82     10372
        1.0       0.65      0.21      0.32      4770

avg / total       0.70      0.72      0.66     15142



In [73]:
## Score the Model on Training and Testing Set
result_scores['SVM'] = \
            (clf.score(X_train,y_train),
             clf.score(X_test,y_test))

In [74]:
get_results(result_scores)


Model                  Train    Test
----------------------------------
Logistic               0.6925   0.7042
Mean                   0.685    0.6953
Logistic_L1            0.6922   0.7046
Logistic_L1_1          0.6924   0.704 
Logistic_L1_C          0.6932   0.704 
Logistic_L1_poly       0.7031   0.7104
RandomForest           0.7284   0.7324
SVM                    0.7158   0.7124
